<a href="https://colab.research.google.com/github/nguyennhatdangkhoa20130295/ML_Thu3_Ca4/blob/main/Lab_8_20130295_NguyenNhatDangKhoa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This lab deals with **GridSearchCV** for tuning the hyper-parameters of an estimator and applying vectorization techniques to the **movie reviews dataset** for classification task. 

*   **Deadline: 23:59, 17/4/2023**



# Import libraries

In [ ]:
# code
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/ML'

Mounted at /content/gdrive
/content/gdrive/MyDrive/ML


In [1]:
# code
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score 
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from prettytable import PrettyTable
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, CategoricalNB, BernoulliNB, MultinomialNB, ComplementNB
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.cluster import KMeans
from scipy.stats import mode
from sklearn.metrics.cluster import completeness_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import fetch_openml
from scipy.cluster.hierarchy import dendrogram, linkage
import scipy.cluster.hierarchy as shc
from sklearn.model_selection import GridSearchCV

#Task 1. With **iris** dataset
*  1.1. Apply **GridSearchCV** for **SVM** to find the best hyperparameters using the following param_grid.

```
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear']}
```




In [ ]:
#code
dataset = datasets.load_iris()
X = dataset.data
y = dataset.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf','linear']}
svm = SVC(kernel='linear')
grid = GridSearchCV(estimator=svm, param_grid=param_grid,scoring='accuracy', refit=True, return_train_score=True)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

svm_accuracy = accuracy_score(y_test, y_pred)
svm_precision = precision_score(y_test, y_pred, average = "macro")
svm_recall = precision_score(y_test, y_pred, average = "macro")
svm_f1 = f1_score(y_test, y_pred, average = "macro")

t = PrettyTable(['Algorithm','Accuracy', 'Precision', 'Recall', 'F1'])
t.add_row(["SVM", round(svm_accuracy, 2), round(svm_precision, 2), round(svm_recall, 2), round(svm_f1, 2)])

print(t)
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)


+-----------+----------+-----------+--------+------+
| Algorithm | Accuracy | Precision | Recall |  F1  |
+-----------+----------+-----------+--------+------+
|    SVM    |   0.97   |    0.98   |  0.98  | 0.96 |
+-----------+----------+-----------+--------+------+
{'C': 1, 'gamma': 1, 'kernel': 'linear'}
SVC(C=1, gamma=1, kernel='linear')
0.9916666666666668


*  1.2. Apply **GridSearchCV** for **kNN** to find the best hyperparameters using the following param_grid.

```
grid_params = { 'n_neighbors' : [5,7,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
```
where

    *  **n_neighbors**: Decide the best k based on the values we have computed earlier.
    *  **weights**: Check whether adding weights to the data points is beneficial to the model or not. 'uniform' assigns no weight, while 'distance' weighs points by the inverse of their distances meaning nearer points will have more weight than the farther points.
    *  **metric**: The distance metric to be used will calculating the similarity.


In [ ]:
#code
grid_params = { 'n_neighbors' : [5,7,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
knn = KNeighborsClassifier()
grid = GridSearchCV(estimator=knn, param_grid=grid_params,scoring='accuracy', refit=True, return_train_score=True)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

knn_accuracy = accuracy_score(y_test, y_pred)
knn_precision = precision_score(y_test, y_pred, average = "macro")
knn_recall = precision_score(y_test, y_pred, average = "macro")
knn_f1 = f1_score(y_test, y_pred, average = "macro")

t = PrettyTable(['Algorithm','Accuracy', 'Precision', 'Recall', 'F1'])
t.add_row(["kNN", round(knn_accuracy, 2), round(knn_precision, 2), round(knn_recall, 2), round(knn_f1, 2)])

print(t)
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)


+-----------+----------+-----------+--------+------+
| Algorithm | Accuracy | Precision | Recall |  F1  |
+-----------+----------+-----------+--------+------+
|    kNN    |   0.93   |    0.96   |  0.96  | 0.92 |
+-----------+----------+-----------+--------+------+
{'metric': 'manhattan', 'n_neighbors': 13, 'weights': 'uniform'}
KNeighborsClassifier(metric='manhattan', n_neighbors=13)
0.9833333333333334


*  1.3. Apply **GridSearchCV** for **Random Forest** to find the best hyperparameters using the following param_grid.

```
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}
```

In [ ]:
#code
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}
rfc = RandomForestClassifier(n_estimators=100)
grid = GridSearchCV(estimator=rfc, param_grid=param_grid,scoring='accuracy', refit=True, return_train_score=True)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

rfc_accuracy = accuracy_score(y_test, y_pred)
rfc_precision = precision_score(y_test, y_pred, average = "macro")
rfc_recall = precision_score(y_test, y_pred, average = "macro")
rfc_f1 = f1_score(y_test, y_pred, average = "macro")

t = PrettyTable(['Algorithm','Accuracy', 'Precision', 'Recall', 'F1'])
t.add_row(["Random Forest", round(rfc_accuracy, 2), round(rfc_precision, 2), round(rfc_recall, 2), round(rfc_f1, 2)])

print(t)
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

+---------------+----------+-----------+--------+------+
|   Algorithm   | Accuracy | Precision | Recall |  F1  |
+---------------+----------+-----------+--------+------+
| Random Forest |   0.93   |    0.96   |  0.96  | 0.92 |
+---------------+----------+-----------+--------+------+
{'max_depth': 3, 'max_features': None, 'max_leaf_nodes': 9, 'n_estimators': 50}
RandomForestClassifier(max_depth=3, max_features=None, max_leaf_nodes=9,
                       n_estimators=50)
0.9833333333333332


*   1.4 Compare the best obtained results from 1.1 to 1.3 (use PrettyTable to dispaly the results)

In [ ]:
# code
t = PrettyTable(['Algorithm','Accuracy', 'Precision', 'Recall', 'F1'])
t.add_row(["SVM", round(svm_accuracy, 2), round(svm_precision, 2), round(svm_recall, 2), round(svm_f1, 2)])
t.add_row(["kNN", round(knn_accuracy, 2), round(knn_precision, 2), round(knn_recall, 2), round(knn_f1, 2)])
t.add_row(["Random forest", round(rfc_accuracy, 2), round(rfc_precision, 2), round(rfc_recall, 2), round(rfc_f1, 2)])

print(t)

+---------------+----------+-----------+--------+------+
|   Algorithm   | Accuracy | Precision | Recall |  F1  |
+---------------+----------+-----------+--------+------+
|      SVM      |   0.97   |    0.98   |  0.98  | 0.96 |
|      kNN      |   0.93   |    0.96   |  0.96  | 0.92 |
| Random forest |   0.93   |    0.96   |  0.96  | 0.92 |
+---------------+----------+-----------+--------+------+


#Task 2. 
For breast cancer dataset (https://tinyurl.com/3vme8hr3) which could be loaded from datasets in sklearn as follows:

```
#Import scikit-learn dataset library
from sklearn import datasets

#Load dataset
cancer = datasets.load_breast_cancer()
```

*   Apply **GridSearchCV** to different classification algorithms such as **SVM, kNN, LogisticRegression, RandomForest**.
*   Compare the results obtained by the best hyperparameters among classification algorithms.

*   2.1. Apply **GridSearchCV** to **SVM** 


In [ ]:
# code
cancer = datasets.load_breast_cancer()
X = cancer.data
y = cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf','linear']}
svm = SVC(kernel='linear')
grid = GridSearchCV(estimator=svm, param_grid=param_grid,scoring='accuracy', refit=True, return_train_score=True)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

svm_accuracy = accuracy_score(y_test, y_pred)
svm_precision = precision_score(y_test, y_pred, average = "macro")
svm_recall = precision_score(y_test, y_pred, average = "macro")
svm_f1 = f1_score(y_test, y_pred, average = "macro")

t = PrettyTable(['Algorithm','Accuracy', 'Precision', 'Recall', 'F1'])
t.add_row(["SVM", round(svm_accuracy, 2), round(svm_precision, 2), round(svm_recall, 2), round(svm_f1, 2)])

print(t)
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

+-----------+----------+-----------+--------+------+
| Algorithm | Accuracy | Precision | Recall |  F1  |
+-----------+----------+-----------+--------+------+
|    SVM    |   0.95   |    0.94   |  0.94  | 0.95 |
+-----------+----------+-----------+--------+------+
{'C': 10, 'gamma': 1, 'kernel': 'linear'}
SVC(C=10, gamma=1, kernel='linear')
0.9589087809036659


*   2.2. Apply **GridSearchCV** to **kNN** 

In [ ]:
#code
grid_params = { 'n_neighbors' : [5,7,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
knn = KNeighborsClassifier()
grid = GridSearchCV(estimator=knn, param_grid=grid_params,scoring='accuracy', refit=True, return_train_score=True)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

knn_accuracy = accuracy_score(y_test, y_pred)
knn_precision = precision_score(y_test, y_pred, average = "macro")
knn_recall = precision_score(y_test, y_pred, average = "macro")
knn_f1 = f1_score(y_test, y_pred, average = "macro")

t = PrettyTable(['Algorithm','Accuracy', 'Precision', 'Recall', 'F1'])
t.add_row(["kNN", round(knn_accuracy, 2), round(knn_precision, 2), round(knn_recall, 2), round(knn_f1, 2)])

print(t)
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

+-----------+----------+-----------+--------+------+
| Algorithm | Accuracy | Precision | Recall |  F1  |
+-----------+----------+-----------+--------+------+
|    kNN    |   0.96   |    0.96   |  0.96  | 0.96 |
+-----------+----------+-----------+--------+------+
{'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'uniform'}
KNeighborsClassifier(metric='manhattan')
0.9120630861040068


*   2.3. Apply **GridSearchCV** to **LogisticRegression** 

In [ ]:
#code
lr = LogisticRegression()
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2'],
    'max_iter': [5000,6000],
    'solver': ['liblinear', 'saga']}
grid = GridSearchCV(estimator=lr, param_grid=param_grid,scoring='accuracy', refit=True, return_train_score=True)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

lr_accuracy = accuracy_score(y_test, y_pred)
lr_precision = precision_score(y_test, y_pred, average = "macro")
lr_recall = precision_score(y_test, y_pred, average = "macro")
lr_f1 = f1_score(y_test, y_pred, average = "macro")

t = PrettyTable(['Algorithm','Accuracy', 'Precision', 'Recall', 'F1'])
t.add_row(["Logistic Regression", round(lr_accuracy, 2), round(lr_precision, 2), round(lr_recall, 2), round(lr_f1, 2)])

print(t)
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

+---------------------+----------+-----------+--------+------+
|      Algorithm      | Accuracy | Precision | Recall |  F1  |
+---------------------+----------+-----------+--------+------+
| Logistic Regression |   0.97   |    0.97   |  0.97  | 0.97 |
+---------------------+----------+-----------+--------+------+
{'C': 100, 'max_iter': 5000, 'penalty': 'l1', 'solver': 'liblinear'}
LogisticRegression(C=100, max_iter=5000, penalty='l1', solver='liblinear')
0.9503410059676044


*   2.4. Apply **GridSearchCV** to **RandomForest** 

In [ ]:
#code
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}
rfc = RandomForestClassifier(n_estimators=100)
grid = GridSearchCV(estimator=rfc, param_grid=param_grid,scoring='accuracy', refit=True, return_train_score=True)
grid.fit(X_train, y_train)
y_pred = grid.predict(X_test)

rfc_accuracy = accuracy_score(y_test, y_pred)
rfc_precision = precision_score(y_test, y_pred, average = "macro")
rfc_recall = precision_score(y_test, y_pred, average = "macro")
rfc_f1 = f1_score(y_test, y_pred, average = "macro")

t = PrettyTable(['Algorithm','Accuracy', 'Precision', 'Recall', 'F1'])
t.add_row(["Random Forest", round(rfc_accuracy, 2), round(rfc_precision, 2), round(rfc_recall, 2), round(rfc_f1, 2)])

print(t)
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

+---------------+----------+-----------+--------+------+
|   Algorithm   | Accuracy | Precision | Recall |  F1  |
+---------------+----------+-----------+--------+------+
| Random Forest |   0.96   |    0.96   |  0.96  | 0.96 |
+---------------+----------+-----------+--------+------+
{'max_depth': 9, 'max_features': 'log2', 'max_leaf_nodes': 9, 'n_estimators': 25}
RandomForestClassifier(max_depth=9, max_features='log2', max_leaf_nodes=9,
                       n_estimators=25)
0.9559249786871271


*   2.5. Compare the best obtained results among classification algorithms (use PrettyTable to dispaly the results) 

In [ ]:
#code
t = PrettyTable(['Algorithm','Accuracy', 'Precision', 'Recall', 'F1'])
t.add_row(["SVM", round(svm_accuracy, 2), round(svm_precision, 2), round(svm_recall, 2), round(svm_f1, 2)])
t.add_row(["kNN", round(knn_accuracy, 2), round(knn_precision, 2), round(knn_recall, 2), round(knn_f1, 2)])
t.add_row(["Logistic Regression", round(lr_accuracy, 2), round(lr_precision, 2), round(lr_recall, 2), round(lr_f1, 2)])
t.add_row(["Random Forest", round(rfc_accuracy, 2), round(rfc_precision, 2), round(rfc_recall, 2), round(rfc_f1, 2)])

print(t)

+---------------------+----------+-----------+--------+------+
|      Algorithm      | Accuracy | Precision | Recall |  F1  |
+---------------------+----------+-----------+--------+------+
|         SVM         |   0.95   |    0.94   |  0.94  | 0.95 |
|         kNN         |   0.96   |    0.96   |  0.96  | 0.96 |
| Logistic Regression |   0.97   |    0.97   |  0.97  | 0.97 |
|    Random Forest    |   0.96   |    0.96   |  0.96  | 0.96 |
+---------------------+----------+-----------+--------+------+


#Task 3. 
The dataset consists of **2000 user-created movie reviews** archived on the IMDb(Internet Movie Database). The reviews are equally partitioned into a positive set and a negative set (1000+1000). Each review consists of a plain text file (.txt) and a class label representing the overall user opinion. 
The class attribute has only two values: **pos** (positive) or **neg** (negative).


*   3.1 Importing additional libraries

In [2]:
import nltk, random
nltk.download('movie_reviews')#download movie reviews dataset
from nltk.corpus import movie_reviews
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score
from collections import Counter
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.


*   3.2. Movie reviews information

In [3]:
#code
print(len(movie_reviews.fileids()))
print(movie_reviews.categories())
print(movie_reviews.words()[:100])
print(movie_reviews.fileids()[:10])

2000
['neg', 'pos']
['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...]
['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt', 'neg/cv005_29357.txt', 'neg/cv006_17022.txt', 'neg/cv007_4992.txt', 'neg/cv008_29326.txt', 'neg/cv009_29417.txt']


*   3.3. Create dataset from movie reviews

In [4]:
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.seed(123)
random.shuffle(documents)

In [5]:
print('Number of Reviews/Documents: {}'.format(len(documents)))
print('Corpus Size (words): {}'.format(np.sum([len(d) for (d,l) in documents])))
print('Sample Text of Doc 1:')
print('-'*30)
print(' '.join(documents[0][0][:50])) # first 50 words of the first document

Number of Reviews/Documents: 2000
Corpus Size (words): 1583820
Sample Text of Doc 1:
------------------------------
most movies seem to release a third movie just so it can be called a trilogy . rocky iii seems to kind of fit in that category , but manages to be slightly unique . the rocky formula of " rocky loses fight / rocky trains / rocky wins fight


In [6]:
sentiment_distr = Counter([label for (words, label) in documents])
print(sentiment_distr)

Counter({'pos': 1000, 'neg': 1000})


*   3.4. Train test split

In [7]:
train, test = train_test_split(documents, test_size = 0.33, random_state=42)

In [8]:
## Sentiment Distrubtion for Train and Test
print(Counter([label for (words, label) in train]))
print(Counter([label for (words, label) in test]))

Counter({'neg': 674, 'pos': 666})
Counter({'pos': 334, 'neg': 326})


In [10]:
X_train = [' '.join(words) for (words, label) in train]
X_test = [' '.join(words) for (words, label) in test]
y_train = [label for (words, label) in train]
y_test = [label for (words, label) in test]

*   3.5. Text Vectorization

In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tfidf_vec = TfidfVectorizer(min_df = 10, token_pattern = r'[a-zA-Z]+')
X_train_bow = tfidf_vec.fit_transform(X_train) # fit train
X_test_bow = tfidf_vec.transform(X_test) # transform test

*   3.6. Apply **SVM** with **GridSearchCV** 

In [ ]:
#code
param_grid = {'C': [0.5, 20, 100, 500],
              'gamma': [1, 0.1, 0.01, 0.001],
              'kernel': ['rbf','linear']}
svm = SVC(kernel='linear')
grid = GridSearchCV(estimator=svm, param_grid=param_grid,scoring='accuracy', refit=True, return_train_score=True)
grid.fit(X_train_bow, y_train)
y_pred = grid.predict(X_test_bow)

svm_accuracy = accuracy_score(y_test, y_pred)
svm_precision = precision_score(y_test, y_pred, average = "macro")
svm_recall = precision_score(y_test, y_pred, average = "macro")
svm_f1 = f1_score(y_test, y_pred, average = "macro")

t = PrettyTable(['Algorithm','Accuracy', 'Precision', 'Recall', 'F1'])
t.add_row(["SVM", round(svm_accuracy, 2), round(svm_precision, 2), round(svm_recall, 2), round(svm_f1, 2)])

print(t)
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

+-----------+----------+-----------+--------+------+
| Algorithm | Accuracy | Precision | Recall |  F1  |
+-----------+----------+-----------+--------+------+
|    SVM    |   0.81   |    0.81   |  0.81  | 0.81 |
+-----------+----------+-----------+--------+------+
{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}
SVC(C=100, gamma=0.01)
0.8492537313432835


*   3.7. Apply **RandomForest** with **GridSearchCV** 

In [ ]:
#code
param_grid = {
    'n_estimators': [25, 50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}
rfc = RandomForestClassifier(n_estimators=100)
grid = GridSearchCV(estimator=rfc, param_grid=param_grid,scoring='accuracy', refit=True, return_train_score=True)
grid.fit(X_train_bow, y_train)
y_pred = grid.predict(X_test_bow)

rfc_accuracy = accuracy_score(y_test, y_pred)
rfc_precision = precision_score(y_test, y_pred, average = "macro")
rfc_recall = precision_score(y_test, y_pred, average = "macro")
rfc_f1 = f1_score(y_test, y_pred, average = "macro")

t = PrettyTable(['Algorithm','Accuracy', 'Precision', 'Recall', 'F1'])
t.add_row(["Random Forest", round(rfc_accuracy, 2), round(rfc_precision, 2), round(rfc_recall, 2), round(rfc_f1, 2)])

print(t)
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

+---------------+----------+-----------+--------+-----+
|   Algorithm   | Accuracy | Precision | Recall |  F1 |
+---------------+----------+-----------+--------+-----+
| Random Forest |   0.8    |    0.8    |  0.8   | 0.8 |
+---------------+----------+-----------+--------+-----+
{'max_depth': 6, 'max_features': 'sqrt', 'max_leaf_nodes': 9, 'n_estimators': 150}
RandomForestClassifier(max_depth=6, max_leaf_nodes=9, n_estimators=150)
0.7932835820895523


*   3.8. Apply **kNN** with **GridSearchCV** 

In [12]:
#code
grid_params = { 'n_neighbors' : [5,7,9,11,13,15],
               'weights' : ['uniform','distance'],
               'metric' : ['minkowski','euclidean','manhattan']}
knn = KNeighborsClassifier()
grid = GridSearchCV(estimator=knn, param_grid=grid_params,scoring='accuracy', refit=True, return_train_score=True)
grid.fit(X_train_bow, y_train)
y_pred = grid.predict(X_test_bow)

knn_accuracy = accuracy_score(y_test, y_pred)
knn_precision = precision_score(y_test, y_pred, average = "macro")
knn_recall = precision_score(y_test, y_pred, average = "macro")
knn_f1 = f1_score(y_test, y_pred, average = "macro")

t = PrettyTable(['Algorithm','Accuracy', 'Precision', 'Recall', 'F1'])
t.add_row(["kNN", round(knn_accuracy, 2), round(knn_precision, 2), round(knn_recall, 2), round(knn_f1, 2)])

print(t)
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

KeyboardInterrupt: ignored

*   3.9. Apply **LogisticRegression** with **GridSearchCV** 

In [ ]:
#code
lr = LogisticRegression()
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'penalty': ['l1', 'l2'],
    'max_iter': [5000,6000],
    'solver': ['liblinear', 'saga']}
grid = GridSearchCV(estimator=lr, param_grid=param_grid,scoring='accuracy', refit=True, return_train_score=True)
grid.fit(X_train_bow, y_train)
y_pred = grid.predict(X_test_bow)

lr_accuracy = accuracy_score(y_test, y_pred)
lr_precision = precision_score(y_test, y_pred, average = "macro")
lr_recall = precision_score(y_test, y_pred, average = "macro")
lr_f1 = f1_score(y_test, y_pred, average = "macro")

t = PrettyTable(['Algorithm','Accuracy', 'Precision', 'Recall', 'F1'])
t.add_row(["Logistic Regression", round(lr_accuracy, 2), round(lr_precision, 2), round(lr_recall, 2), round(lr_f1, 2)])

print(t)
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

*   3.10. Compare the best obtained results among classification algorithms (use PrettyTable to dispaly the results) 

In [ ]:
# code
t = PrettyTable(['Algorithm','Accuracy', 'Precision', 'Recall', 'F1'])
t.add_row(["SVM", round(svm_accuracy, 2), round(svm_precision, 2), round(svm_recall, 2), round(svm_f1, 2)])
t.add_row(["kNN", round(knn_accuracy, 2), round(knn_precision, 2), round(knn_recall, 2), round(knn_f1, 2)])
t.add_row(["Logistic Regression", round(lr_accuracy, 2), round(lr_precision, 2), round(lr_recall, 2), round(lr_f1, 2)])
t.add_row(["Random Forest", round(rfc_accuracy, 2), round(rfc_precision, 2), round(rfc_recall, 2), round(rfc_f1, 2)])

print(t)

#Finally,
Save a copy in your Github. Remember renaming the notebook.